In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        


In [ ]:
df=pd.read_csv(os.path.join(dirname,'covid_19_data.csv'))

In [ ]:
df

In [ ]:
df['Country/Region'].unique()

In [ ]:
x=df[df['Country/Region']=='India']

In [ ]:
x

In [ ]:
x['Last Update'] = pd.to_datetime(x['Last Update'])

In [ ]:
x.head()

In [ ]:
#x.groupby(x['Last Update'])

In [ ]:
preprocessed_x=x.groupby(x['Last Update'].dt.date)['Confirmed','Deaths','Recovered'].agg(['sum'])

In [ ]:
preprocessed_x

In [ ]:
preprocessed_x.iloc[:,0]

In [ ]:
preprocessed_x.index

Confirmed-Recovered -Deaths Plot

In [ ]:
xplot=preprocessed_x.index
y_confirmed=preprocessed_x.iloc[:,0].values
y_deaths=preprocessed_x.iloc[:,1].values
y_recovered=preprocessed_x.iloc[:,2].values

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(xplot,y_confirmed)

plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(xplot,y_recovered)
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(xplot,y_deaths)
plt.show()

Using the historical day Predicting the deaths ,confirmed cases and recovered on that day

In [ ]:
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.models import load_model

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)
# fix random seed for reproducibility
np.random.seed(7)
# load the dataset

In [ ]:
X=[]
for i in range(len(xplot)):
  X.append(i)

CONFIRMED TIME SERIES ANALYSIS
INDIA


In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
y_confirmed = scaler.fit_transform(y_confirmed.reshape(-1,1))
# split into train and test sets
train_size = int(len(y_confirmed) * 0.67)
test_size = len(y_confirmed) - train_size
trainX,testX,trainY,testY=X[0:train_size],X[train_size:],y_confirmed[0:train_size],y_confirmed[train_size:0]

In [ ]:
look_back = 20
trainX, trainY = create_dataset(trainX, look_back)
testX, testY = create_dataset(testX, look_back)

In [ ]:
print(trainX.shape,trainY.shape,testX.shape)

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [ ]:
model = Sequential()
model.add(LSTM(20, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
history=model.fit(trainX, trainY, epochs=1000, batch_size=1)


In [ ]:
'''es = EarlyStopping(monitor='loss', mode='min', verbose=1,patience=64)
mc = ModelCheckpoint(r'Covid_19_TimeSeries_Confirmed.h5', monitor='accuracy',mode='max', verbose=1, save_best_only=True) 
 # fit mode'''


In [ ]:
plt.plot(history.history['accuracy'])
plt.title('model accuracy vs loss')
plt.ylabel('accuracy')
plt.plot(history.history['loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
plt.show() 

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform(trainY.reshape(-1,1))
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform(testY.reshape(-1,1))

In [ ]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(y_confirmed)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

In [ ]:
testPredictPlot = np.empty_like(y_confirmed)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(y_confirmed)-1, :] = testPredict

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(scaler.inverse_transform(y_confirmed))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.legend(['Actual','Train prediction','Test prediction'])
plt.show()

Deaths Time Series Analaysis india

In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
y_deaths = scaler.fit_transform(y_deaths.reshape(-1,1))
# split into train and test sets
train_size = int(len(y_deaths) * 0.67)
test_size = len(y_deaths) - train_size
trainX,testX,trainY,testY=X[0:train_size],X[train_size:],y_deaths[0:train_size],y_deaths[train_size:0]

In [ ]:
look_back = 20
trainX, trainY = create_dataset(trainX, look_back)
testX, testY = create_dataset(testX, look_back)

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [ ]:
model = Sequential()
model.add(LSTM(20, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])


In [ ]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1,patience=64)
mc = ModelCheckpoint(r'Covid_19_TimeSeries_Deaths.h5', monitor='accuracy',mode='max', verbose=1, save_best_only=True) 
 # fit mode
history=model.fit(trainX, trainY, epochs=1000, batch_size=1, verbose=1,callbacks=[es,mc])

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('model accuracy vs loss')
plt.ylabel('accuracy')
plt.plot(history.history['loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
plt.show() 

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform(trainY.reshape(-1,1))
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform(testY.reshape(-1,1))
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(y_deaths)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = np.empty_like(y_deaths)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(y_deaths)-1, :] = testPredict

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(scaler.inverse_transform(y_deaths))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.legend(['Actual','Train prediction','Test prediction'])
plt.show()

Recovered TimeSeries Analysis India

In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
y_recovered= scaler.fit_transform(y_recovered.reshape(-1,1))
# split into train and test sets
train_size = int(len(y_recovered) * 0.67)
test_size = len(y_recovered) - train_size
trainX,testX,trainY,testY=X[0:train_size],X[train_size:],y_recovered[0:train_size],y_recovered[train_size:0]

In [ ]:
look_back = 20
trainX, trainY = create_dataset(trainX, look_back)
testX, testY = create_dataset(testX, look_back)

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [ ]:
model = Sequential()
model.add(LSTM(20, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])


In [ ]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1,patience=64)
mc = ModelCheckpoint(r'Covid_19_TimeSeries_recovered.h5', monitor='accuracy',mode='max', verbose=1, save_best_only=True) 
 # fit mode
history=model.fit(trainX, trainY, epochs=1000, batch_size=1, verbose=1,callbacks=[es,mc])

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('model accuracy vs loss')
plt.ylabel('accuracy')
plt.plot(history.history['loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
plt.show() 

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform(trainY.reshape(-1,1))
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform(testY.reshape(-1,1))
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(y_recovered)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
testPredictPlot = np.empty_like(y_recovered)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(y_recovered)-1, :] = testPredict

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(scaler.inverse_transform(y_recovered))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.legend(['Actual','Train prediction','Test prediction'])
plt.show()